# Installation et imports

### Installation des modules 

In [418]:
%pip install Pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [419]:
%pip install Matplotlib 


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [420]:
%pip install Scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [421]:
%pip install Numpy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [537]:
%pip install tqdm
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 2.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Import et données

In [422]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib as plt
import math


In [519]:
data = pd.read_csv("../dfClean.csv")

counts = data['descr_grav'].value_counts()

total_0 = counts[0]/len(data)
total_1 = counts[1]/len(data)
total_2 = counts[2]/len(data)
total_3 = counts[3]/len(data)

Proportion_0 = data[data['descr_grav'] == 0].sample(n=int(7500*total_0))
Proportion_1 = data[data['descr_grav'] == 1].sample(n=int(7500*total_1))
Proportion_2 = data[data['descr_grav'] == 2].sample(n=int(7500*total_2))
Proportion_3 = data[data['descr_grav'] == 3].sample(n=int(7500*total_3))

data_reduit = pd.concat([Proportion_0,Proportion_1, Proportion_2, Proportion_3])
data_reduit = data_reduit.reset_index(drop=True)
data_reduit = data_reduit.drop(columns=["latitude", "longitude"])
print(len(data_reduit))
display(data_reduit)

7498


,date,descr_cat_veh,descr_agglo,place,descr_dispo_secu,descr_grav
0,11099,0,0,0,14,0
1,8119,0,0,3,14,0
2,25496,1,1,0,14,0
3,10255,1,1,0,14,0
4,16977,0,0,0,14,0
...,...,...,...,...,...,...
7493,7918,6,1,0,4,3
7494,28858,2,1,0,11,3
7495,7578,0,1,0,14,3
7496,24041,6,0,0,4,3


# Répartition des données

### Répartition holdout 

In [424]:
from sklearn.model_selection import train_test_split

In [507]:
def holdout(dataframe, colonne_predict, num_rep, train_s, test_s ):
        liste_data_train_test = []
        liste_data = []
        clean_data = dataframe.drop(colonne_predict, axis = 1)
        y = dataframe[colonne_predict]
        for i in range(num_rep):
                X_train, X_test, y_train, y_test = train_test_split(clean_data,y, train_size= train_s, test_size = test_s)
                X_train = X_train.reset_index(drop=True)
                y_train = y_train.reset_index(drop=True)
                X_test = X_test.reset_index(drop=True)
                y_test = y_test.reset_index(drop=True)
                liste_data_train_test.append([X_train, X_test, y_train, y_test])
        return liste_data_train_test

### Répartition Leave One Out

In [427]:
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

In [586]:
def leaveoneout(dataframe, colonne_predict,k):
    liste_y_pred = []
    liste_y_test = []
    X = dataframe.drop(colonne_predict, axis = 1) 
    y = dataframe[colonne_predict]
    LOO = LeaveOneOut()
    knn = KNeighborsClassifier(n_neighbors=k)
    
    for train_index, test_index in (LOO.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        liste_y_pred.append(y_pred)
        liste_y_test.append(y_test)
        
    return liste_y_pred, liste_y_test

# Classification avec KNN

### Fonctions from scratch

In [467]:
#Fonction de classification knn
def KnnClassification_distance_euclidienne(ligne,y_train,df,k):
    dictList = [None] * len(df)
    liste_y_pred = []
    for index, row in df.iterrows():
        dictList[index] = np.linalg.norm(ligne - row) #calcul de la distance euclidienne
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    for index,value in sorted_values:
        #firstk.append([df.loc[index]])
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

In [466]:
#Fonction de classification knn
def KnnClassification_distance_manhattan(ligne, y_train, df, k):
    dictList = [None] * len(df)
    liste_y_pred = []
    
    for index, row in df.iterrows():
        dictList[index] = np.sum(np.abs(ligne - row))  # Calcul de la distance de Manhattan
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    
    for index, value in sorted_values:
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

In [464]:
def KnnClassification_distance_sup_norme(ligne, y_train, df, k):
    dictList = [None] * len(df)
    liste_y_pred = []
    
    for index, row in df.iterrows():
        dictList[index] = np.max(np.abs(ligne - row))  # Calcul de la distance de la norme supérieure
    
    sorted_values = sorted(enumerate(dictList), key=lambda x: x[1])[:k]
    
    for index, value in sorted_values:
        liste_y_pred.append(y_train[index])
    
    y_pred = np.bincount(liste_y_pred).argmax()
        
    return y_pred

### Scikit learn

In [575]:
#prediction holdout sklearn
def  KnnClassification_sk_learn(liste_data,k):
    knn = KNeighborsClassifier(n_neighbors=k)
    liste_y_pred_holdout_sk = []
    liste_y_test_holdout_sk = []
    for i in range(len(liste_data)):
        X_train = liste_data[i][0]
        X_test = liste_data[i][1]
        y_train = liste_data[i][2]
        y_test = liste_data[i][3]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        liste_y_pred_holdout_sk.append(y_pred)
        liste_y_test_holdout_sk.append(y_test)
    return liste_y_pred_holdout_sk, liste_y_test_holdout_sk

### Calcul de score

In [447]:
from sklearn.metrics import accuracy_score

In [587]:
#Répartition leave One Out
y_pred_LOO_5, y_test_LOO_5 = leaveoneout(data_reduit, "descr_grav", 5)
y_pred_LOO_7, y_test_LOO_7 = leaveoneout(data_reduit, "descr_grav", 7)
y_pred_LOO_10, y_test_LOO_10 = leaveoneout(data_reduit, "descr_grav", 10)

KeyError: "None of [Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,\n       ...\n       7488, 7489, 7490, 7491, 7492, 7493, 7494, 7495, 7496, 7497],\n      dtype='int64', length=7497)] are in the [columns]"

In [521]:
# Score répartition leave One Out
accuracy_LOO_5 = accuracy_score(y_test_LOO_5, y_pred_LOO_5)
accuracy_LOO_7 = accuracy_score(y_test_LOO_7, y_pred_LOO_7)
accuracy_LOO_10 = accuracy_score(y_test_LOO_10, y_pred_LOO_10)

print(accuracy_LOO_5*100)
print(accuracy_LOO_7*100)
print(accuracy_LOO_10*100)

39.82395305414777
39.7839423846359
40.87756735129368


In [557]:
#Répartition hold out
liste_data_holdout = []
liste_data_holdout  = holdout(data_reduit,"descr_grav", 5, 0.8, 0.2)

# Prediction holdout from scratch
total_y_pred_holdout_euclidienne = []
total_y_pred_holdout_manhattan = []
total_y_pred_holdout_sup_norme = []

X_test_0 = liste_data_holdout[0][1]
y_train_0  = liste_data_holdout[0][2]
X_train_0 = liste_data_holdout[0][0]
y_test_0 =  liste_data_holdout[0][3]

for k in [5,7, 10]:
    liste_y_pred_holdout_euclidienne = []
    liste_y_pred_holdout_manhattan = []
    liste_y_pred_holdout_sup_norme = []
    
    for i in range(len(X_test_0)):
        y_pred_holdout_euclidienne = KnnClassification_distance_euclidienne(X_test_0.loc[i], y_train_0, X_train_0, k)
        liste_y_pred_holdout_euclidienne.append(y_pred_holdout_euclidienne)

        y_pred_holdout_manhattan = KnnClassification_distance_manhattan(X_test_0.loc[i], y_train_0, X_train_0, k)
        liste_y_pred_holdout_manhattan.append(y_pred_holdout_manhattan)

        y_pred_holdout_sup_norme = KnnClassification_distance_sup_norme(X_test_0.loc[i], y_train_0, X_train_0, k)
        liste_y_pred_holdout_sup_norme.append(y_pred_holdout_sup_norme)
        
    
    total_y_pred_holdout_euclidienne.append(liste_y_pred_holdout_euclidienne)
    total_y_pred_holdout_manhattan.append(liste_y_pred_holdout_manhattan)
    total_y_pred_holdout_sup_norme.append(liste_y_pred_holdout_sup_norme)

In [564]:
print(len(y_test_0))
print(len(total_y_pred_holdout_euclidienne[0]))

1500
1500


In [565]:
# Score répartition leave One Out
accuracy_holdout_euclidienne_5 = accuracy_score(y_test_0, total_y_pred_holdout_euclidienne[0])
accuracy_holdout_euclidienne_7 = accuracy_score(y_test_0, total_y_pred_holdout_euclidienne[1])
accuracy_holdout_euclidienne_10 = accuracy_score(y_test_0, total_y_pred_holdout_euclidienne[2])


accuracy_holdout_manhattan_5 = accuracy_score(y_test_0, total_y_pred_holdout_manhattan[0])
accuracy_holdout_manhattan_7 = accuracy_score(y_test_0, total_y_pred_holdout_manhattan[1])
accuracy_holdout_manhattan_10 = accuracy_score(y_test_0, total_y_pred_holdout_manhattan[2])

accuracy_holdout_sup_norme_5 = accuracy_score(y_test_0, total_y_pred_holdout_sup_norme[0])
accuracy_holdout_sup_norme_7 = accuracy_score(y_test_0, total_y_pred_holdout_sup_norme[1])
accuracy_holdout_sup_norme_10 = accuracy_score(y_test_0, total_y_pred_holdout_sup_norme[2])

print("Score euclidienne:", accuracy_holdout_euclidienne_5*100, accuracy_holdout_euclidienne_7*100, accuracy_holdout_euclidienne_10*100)
print("Score manhattan:", accuracy_holdout_manhattan_5*100, accuracy_holdout_manhattan_7*100, accuracy_holdout_manhattan_10*100)
print("Score sup norme:", accuracy_holdout_sup_norme_5*100, accuracy_holdout_sup_norme_7*100, accuracy_holdout_sup_norme_10*100 )

Score euclidienne: 39.06666666666666 39.53333333333333 39.86666666666667
Score manhattan: 40.2 40.8 41.86666666666667
Score sup norme: 38.2 38.53333333333334 39.4


In [584]:
#Score prediction holdout scikit learn
liste_data_holdout  = holdout(data_reduit,"descr_grav", 5, 0.8, 0.2)
liste_accuracy_holdout_sk = []
for k in [5,7,10]:
    liste_y_pred_holdout_sk, liste_y_test_holdout_sk = KnnClassification_sk_learn(liste_data_holdout,k)
    liste_score =[]
    for i in range(5):
        accuracy_holdout_sk = accuracy_score(liste_y_test_holdout_sk[i], liste_y_pred_holdout_sk[i])
        liste_score.append(accuracy_holdout_sk)
    liste_accuracy_holdout_sk.append(liste_score)

In [585]:
print(liste_accuracy_holdout_sk)

[[0.37733333333333335, 0.41933333333333334, 0.3973333333333333, 0.38866666666666666, 0.41133333333333333], [0.376, 0.4086666666666667, 0.38666666666666666, 0.37866666666666665, 0.38666666666666666], [0.39066666666666666, 0.41, 0.394, 0.3873333333333333, 0.414]]


# Évaluation quantitative des résultats

### Évaluation sur les données d'apprentissage

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [588]:
def evaluation(objet, X, Y):
    #calcul taux de classification
    taux_classification = cross_val_score(objet, X, Y, scoring='accuracy')
    #Moyenne taux de classification
    moyenne_taux_classification = np.mean(taux_classification)
    #prédiction 
    Y_pred = cross_val_predict(objet, X, Y)
    #Calcul du score (taux d'apprentissage)
    accuracy = accuracy_score(Y, Y_pred)
    # Matrice de confusion à partir des classes réelle et celles prédites
    matrice_confusion = ConfusionMatrixDisplay.from_predictions(Y, Y_pred, normalize="true",  values_format=".0%") #résultat en pourcentage
    # Précision du modèle
    precision = precision_score(Y, Y_pred, average='weighted')
    # Rappel du modèle
    rappel = recall_score(Y, Y_pred, average='weighted')

    return taux_classification, moyenne_taux_classification, Y_pred, accuracy, matrice_confusion, precision, rappel



### Évaluation sur les données de test